In [2]:
import utils
import numpy as np

image_files, labels = utils.getprocessedfilelist()
np.save(os.path.join('data_road', 'images.npy'), image_files)
np.save(os.path.join('data_road', 'labels.npy'), labels)

In [1]:
import utils
import numpy as np

# use this cell to load the numpy data from disk so that you
# don't always have to parse the file structure
image_files = np.load(os.path.join('data_road', 'images.npy'))
labels = np.load(os.path.join('data_road', 'labels.npy'))

In [2]:
import tensorflow as tf
data = tf.data.Dataset.from_tensor_slices((image_files, labels))
a = data.as_numpy_iterator()
b = a.next()
print(b)

(b'data_road\\processed_images\\umm_000000_0.jpg', array([160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,


In [3]:
import numpy as np
import tensorflow as tf

@tf.function
def readimage(image_file, label):
    image = tf.io.decode_jpeg(tf.io.read_file(image_file)) / 255
    return image, label

shuffled_ds = data.shuffle(len(labels)).map(readimage, num_parallel_calls=tf.data.AUTOTUNE)

iter_ = shuffled_ds.as_numpy_iterator()
a = iter_.next()
print(a)

(array([[[0.13333334, 0.16078432, 0.2       ],
        [0.26666668, 0.2627451 , 0.25490198],
        [0.7058824 , 0.62352943, 0.5411765 ],
        ...,
        [1.        , 0.99607843, 0.972549  ],
        [1.        , 0.95686275, 0.9137255 ],
        [0.80784315, 0.7058824 , 0.654902  ]],

       [[0.20392157, 0.24705882, 0.27058825],
        [0.31764707, 0.32156864, 0.3019608 ],
        [0.7058824 , 0.627451  , 0.53333336],
        ...,
        [1.        , 0.99607843, 0.972549  ],
        [1.        , 0.96862745, 0.92941177],
        [0.85490197, 0.76862746, 0.7176471 ]],

       [[0.21568628, 0.3019608 , 0.2901961 ],
        [0.3647059 , 0.39607844, 0.3529412 ],
        [0.7372549 , 0.67058825, 0.56078434],
        ...,
        [1.        , 1.        , 0.972549  ],
        [1.        , 0.99215686, 0.9529412 ],
        [0.87058824, 0.8156863 , 0.7647059 ]],

       ...,

       [[0.9843137 , 1.        , 1.        ],
        [0.9882353 , 1.        , 1.        ],
        [1.        , 

In [15]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Conv1D, LSTM, MaxPool2D, Flatten, InputLayer, Reshape, TimeDistributed
import tensorflow as tf
import os

def create_model():
    model = Sequential([
        InputLayer(input_shape=(160,600,3)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        TimeDistributed(LSTM(64, return_sequences=True)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        TimeDistributed(LSTM(64, return_sequences=True)),
        Conv2D(64, (5,1), strides=(4,1), activation='relu'),
        Conv2D(1, (4,1), activation='relu'),
        Flatten(),
        Dense(600, activation='linear')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

callbacks = [
    tf.keras.callbacks.TensorBoard(),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join('ckpt','training-1'), 
        save_weights_only=True)
]

batch_size = 32

train_ds = shuffled_ds.take(20000).batch(batch_size)
val_ds = shuffled_ds.skip(20000).take(5000).batch(batch_size)
model = create_model()


In [17]:
tf.keras.backend.clear_session()

model.fit(x=train_ds, batch_size=batch_size, epochs=80, callbacks=callbacks, validation_data=val_ds, verbose=1)

Epoch 1/80
625/625 [==============================] - 192s 287ms/step - loss: 19883.3926 - accuracy: 0.0017 - val_loss: 19863.1465 - val_accuracy: 6.0000e-04
Epoch 2/80
625/625 [==============================] - 167s 267ms/step - loss: 19832.4531 - accuracy: 4.0000e-04 - val_loss: 19800.1113 - val_accuracy: 6.0000e-04
Epoch 3/80
625/625 [==============================] - 162s 259ms/step - loss: 19640.5293 - accuracy: 4.0000e-04 - val_loss: 19455.3242 - val_accuracy: 0.0010
Epoch 4/80
625/625 [==============================] - 158s 253ms/step - loss: 19448.8340 - accuracy: 0.0040 - val_loss: 19468.6992 - val_accuracy: 0.0112
Epoch 5/80
625/625 [==============================] - 159s 255ms/step - loss: 19281.8730 - accuracy: 0.3920 - val_loss: 19094.4199 - val_accuracy: 0.6608
Epoch 6/80
625/625 [==============================] - 158s 253ms/step - loss: 19147.3047 - accuracy: 0.2991 - val_loss: 19074.7344 - val_accuracy: 0.0122
Epoch 7/80
625/625 [==============================] - 162s 2